# VacationPy
----

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("Output/city_weather.csv")
df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,shasta lake,us,40.6804,-122.3708,86.72,41,100,4.23,1627282088
1,punta arenas,cl,-53.1500,-70.9167,35.71,87,0,11.50,1627281935
2,thompson,ca,55.7435,-97.8558,41.16,87,40,0.00,1627281980
3,nikolskoye,ru,59.7035,30.7861,75.09,40,6,4.05,1627281969
4,port alfred,za,-33.5906,26.8910,63.77,30,43,12.75,1627281741
...,...,...,...,...,...,...,...,...,...
577,hailar,cn,49.2000,119.7000,67.30,77,100,11.18,1627282342
578,nanyamba,tz,-10.6833,39.8333,75.04,43,49,11.90,1627282342
579,nsanje,mw,-16.9200,35.2620,66.74,69,63,7.07,1627282343
580,tucumcari,us,35.1717,-103.7250,75.04,60,1,4.61,1627282307


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.__version__

gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lng"]].astype(float)

# Humidity as weight
humidity = df["Humidity"].astype(float)

# Create a Humidity Heatmap layer
morocco_coordinates = (31.43939795909406, -6.103582692418146)

fig = gmaps.figure(center = morocco_coordinates, zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create DataFrame for the ideal weather conditions
# Max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

ideal_weather = df.loc[(df["Max Temp"]>70) & (df["Max Temp"]<80) & (df["Wind Speed"]<10) & (df["Cloudiness"]==0)]
ideal_weather

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
68,joensuu,fi,62.6012,29.7632,71.10,52,0,5.75,1627282112
89,soto la marina,mx,23.7667,-98.2167,74.12,93,0,3.51,1627282120
107,mizdah,ly,31.4451,12.9801,78.44,40,0,7.87,1627282128
241,manaus,br,-3.1019,-60.0250,79.29,78,0,3.44,1627282185
355,villamartin,es,36.8598,-5.6449,73.00,85,0,4.81,1627282237
406,bykivka,ua,50.2929,27.9831,75.25,54,0,9.13,1627282261
460,swinoujscie,pl,53.9105,14.2471,73.54,68,0,5.75,1627282286
469,spornoye,ru,50.8448,37.2913,77.02,43,0,6.93,1627282291
491,skelleftea,se,64.7507,20.9528,70.16,64,0,3.44,1627282301
508,tiznit,ma,29.5833,-9.5000,70.92,52,0,1.72,1627282309


### Hotel Map
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame to hold info about hotels
hotel_df = pd.DataFrame(ideal_weather)
hotel_df["Country"] = hotel_df["Country"].astype(str).str.upper()
hotel_df["Hotel Name"] = ""

# Parameters to query Google places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

target_search = "hotel"
target_radius = 5000

# Define list of indices of cities for which hotels are not found
indices = []

# Iterate over hotel_df to populate Hotel Name
for index,row in hotel_df.iterrows():
    lat = hotel_df.loc[index,"Lat"]
    lng = hotel_df.loc[index,"Lng"]
    
    params = {
        "location": f"{lat},{lng}",
        "radius": target_radius,
        "keyword": target_search,
        "key": g_key
    }
    
    response = requests.get(base_url, params=params).json()
    
    print("----------------------------------------------")
    print(f"Finding the most prominent hotel in the {hotel_df.loc[index,'City']} city ")
   
    try:
        name = response["results"][0]["name"]      
        
        hotel_df.loc[index,"Hotel Name"] = name
        
    except:
        print(f"Within {target_radius} metres Hotel is not found")
        indices.append(index)

hotels_ideal_weather = hotel_df.drop(indices, inplace = False)
hotels_ideal_weather

----------------------------------------------
Finding the most prominent hotel in the joensuu city 
----------------------------------------------
Finding the most prominent hotel in the soto la marina city 
----------------------------------------------
Finding the most prominent hotel in the mizdah city 
----------------------------------------------
Finding the most prominent hotel in the manaus city 
----------------------------------------------
Finding the most prominent hotel in the villamartin city 
Within 5000 metres Hotel is not found
----------------------------------------------
Finding the most prominent hotel in the bykivka city 
Within 5000 metres Hotel is not found
----------------------------------------------
Finding the most prominent hotel in the swinoujscie city 
----------------------------------------------
Finding the most prominent hotel in the spornoye city 
Within 5000 metres Hotel is not found
----------------------------------------------
Finding the most 

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
68,joensuu,FI,62.6012,29.7632,71.10,52,0,5.75,1627282112,"Original Sokos Hotel Vaakuna, Joensuu"
89,soto la marina,MX,23.7667,-98.2167,74.12,93,0,3.51,1627282120,Hacienda
107,mizdah,LY,31.4451,12.9801,78.44,40,0,7.87,1627282128,فندق مزدة الكبير
241,manaus,BR,-3.1019,-60.0250,79.29,78,0,3.44,1627282185,Hotel Villa Amazônia
460,swinoujscie,PL,53.9105,14.2471,73.54,68,0,5.75,1627282286,Hilton Swinoujscie Resort & Spa
491,skelleftea,SE,64.7507,20.9528,70.16,64,0,3.44,1627282301,Scandic Skellefteå
555,jalu,LY,29.0331,21.5482,79.83,49,0,7.45,1627282331,إستراحة صلاح الصادق


In [7]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotels_ideal_weather.iterrows()]
locations = hotels_ideal_weather[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))